### Setup

1. Setup conda env
2. install setuptools
3. install https://github.com/hectornieto/pyPro4Sail a dependency
4. install https://github.com/hectornieto/pyMETRIC the ET model
5. install https://github.com/khufkens/daymetpy for ancillary inputs

Example reference using Daymet for ancillary inputs to estimate ET with Landsat:https://www.sciencedirect.com/science/article/pii/S0034425715302650

projection info for daymet: https://wiki.cyverse.org/wiki/display/~tyson_swetnam/Raster+Calculations

To get daymet data for a 512x512 scene and build an array it will take 235 api calls which may not be too bad.

This func will get the lat, lon tuples of a raster to be used to get daymet data: https://rasterio.readthedocs.io/en/stable/api/rasterio.transform.html

I've chosen daymet over mesonet because it is easier to acquire, but later spatially interpolated Mesonet and other data sources can be used for better accuracy. [This paper](https://www.sciencedirect.com/science/article/pii/S1161030116302076#bib0005) notes:"While, on average, simulated yields using Daymet were in closer agreement with simulations based on MWD compared with PRISM (RMSE%: 18 vs. 24%), there were still large differences in simulated yield in 20% of the site-years. And, perhaps more importantly, these differences were not predictable as they were not associated with any spatial pattern in weather, topography, or weather network. While GWD might be useful for applications that only require temperature, such as crop stage prediction or quantification of early frost risk, water deficit and simulated yields for specific site-years are highly uncertain and there is no way to ex-ante predict the magnitude and direction of the bias, which undermines utility of GWD for field-specific or real-time agronomic applications."

Daymet overview: https://daymet.ornl.gov/overview

The variables we don't have and which are most difficult to get probably are related to wind speed and vegetation structure:
u, z_0M, d_0, z_u. Maybe we get these from mesonet stations, need to look at the literature to see what folks do about this.

In [9]:
import sys
import daymetpy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

denver_loc = (-104.9903, 39.7392)
miami_loc = (-80.2089, 25.7753)

denver = daymetpy.daymet_timeseries(lon=denver_loc[0], lat=denver_loc[1], start_year=2012, end_year=2014)

In [10]:
denver

,year,yday,dayl,prcp,srad,swe,tmax,tmin,vp
2012-01-01,2012.0,1.0,33177.601562,0.0,256.000000,4.0,8.5,-7.0,320.0
2012-01-02,2012.0,2.0,33177.601562,0.0,230.399994,4.0,5.0,-8.0,320.0
2012-01-03,2012.0,3.0,33523.199219,0.0,278.399994,4.0,13.0,-5.0,320.0
2012-01-04,2012.0,4.0,33523.199219,0.0,291.200012,4.0,15.0,-5.0,320.0
2012-01-05,2012.0,5.0,33523.199219,0.0,284.799988,4.0,17.0,-2.0,360.0
...,...,...,...,...,...,...,...,...,...
2014-12-27,2014.0,361.0,33177.601562,0.0,227.199997,8.0,-3.0,-15.5,200.0
2014-12-28,2014.0,362.0,33177.601562,0.0,259.200012,8.0,0.5,-15.0,200.0
2014-12-29,2014.0,363.0,33177.601562,0.0,259.200012,8.0,1.5,-14.0,200.0
2014-12-30,2014.0,364.0,33177.601562,3.0,147.199997,8.0,-10.0,-20.0,120.0


In [14]:
import pyMETRIC.METRIC as met

 Parameters
        ----------
        CHECK (landsat) Tr_K : float
            Radiometric composite temperature (Kelvin).
        CHECK (daymet convert to Kelvin) T_A_K : float
            Air temperature (Kelvin).
        (assume? mesonet stations?) u : float
            Wind speed above the canopy (m s-1).
        CHECK (daymet needs to be converted from pascals to mb) ea : float
            Water vapour pressure above the canopy (mb).
        CHECK (will use default or calc from elevation) p : float
            Atmospheric pressure (mb), use 1013 mb by default.
        CHECK (daymet, right units in W/m/-2 already, but also consider pyMETRIC suggested method from albedo...) S_n : float
            Solar irradiance (W m-2).
        (estimated from vapor pressure and air temp (to get atm emissivity)) L_dn : float
            Downwelling longwave radiation (W m-2)
        (https://modis.gsfc.nasa.gov/data/dataprod/mod11.php) emis : float
            Surface emissivity.
        z_0M : float
            Aerodynamic surface roughness length for momentum transfer (m).
        d_0 : float
            Zero-plane displacement height (m).
        z_u : float
            Height of measurement of windspeed (m).
        CHECK (see daymet docs) z_T : float
            Height of measurement of air temperature (m).
        cold_pixel : tuple
            pixel coordinates (row, col) for the cold endmember
        hot_pixel : tuple
            pixel coordinates (row, col) for the hot endmember
        calcG_params : list[list,float or array], optional
            Method to calculate soil heat flux,parameters.

Also these params are needed for exhaustive hot/cold pixel search, easily computed from Landsat and MODIS for albedo. Or possibly Landsat? this is what EEflux did to estiamte albedo

"Albedo – This represents total, integrated reflectance across the electromagnetic spectrum. It is calculated from the six primary short-wave bands and is useful for energy balance work. Based on Tasumi et al., 2008, ASCE J. Hydrologic Engineering."

    vi_array : numpy array
        Vegetation Index array (-)
    lst_array : numpy array
        Land Surface Temperature array (Kelvin)
    cv_ndvi : numpy array
        Coefficient of variation of Vegetation Index as homogeneity measurement
        from neighboring pixels
    std_lst : numpy array
        Standard deviation of LST as homogeneity measurement
        from neighboring pixels
    cv_albedo : numpy array
        Coefficient of variation of albdeo as homogeneity measurement
        from neighboring pixels

[Good overview of METRIC](https://onlinelibrary.wiley.com/doi/full/10.1111/jawr.12056)

In [ ]:
import pyMETRIC.METRIC as METRIC 
output=METRIC.METRIC(Tr_K, T_A_K, u, ea, p, Sn, L_dn, emis, z_0M, d_0, z_u, z_T, cold_pixel, hot_pixel, LE_cold)

In [15]:
help(met)

Help on module pyMETRIC.METRIC in pyMETRIC:

NAME
    pyMETRIC.METRIC - Created on Sat May 30 10:59:35 2015

DESCRIPTION
    @author: hector

FUNCTIONS
    METRIC(Tr_K, T_A_K, u, ea, p, Sn, L_dn, emis, z_0M, d_0, z_u, z_T, cold_pixel, hot_pixel, LE_cold, LE_hot=0, use_METRIC_resistance=True, calcG_params=[[1], 0.35], UseL=False, UseDEM=False)
        Calulates bulk fluxes using METRIC model
        
        Parameters
        ----------
        Tr_K : float
            Radiometric composite temperature (Kelvin).
        T_A_K : float
            Air temperature (Kelvin).
        u : float
            Wind speed above the canopy (m s-1).
        ea : float
            Water vapour pressure above the canopy (mb).
        p : float
            Atmospheric pressure (mb), use 1013 mb by default.
        S_n : float
            Solar irradiance (W m-2).
        L_dn : float
            Downwelling longwave radiation (W m-2)
        emis : float
            Surface emissivity.
        z_0M : 